#TODO : Add a description of the notebook
#TODO : Fix part 5 production plot (i.e. the inclusion of storage)
#TODO : Add a few plots

# Imports area

In [ ]:
import os
from pathlib import Path

# Check if the current directory is named "eu_7_nodes"
current_path = Path.cwd()
if current_path.name == "eu_7_nodes":
    # Go two directories back
    new_path = current_path.parent.parent
    print(f"Changing to directory: {new_path}")
    
    # Change the current directory
    os.chdir(new_path)
    print(f"Current directory after change: {Path.cwd()}")

else:
    print("Current directory is not 'eu_7_nodes'. No change needed.")
    print(f"Current directory: {current_path}")


In [ ]:
import os
import sys
import pandas as pd
import linopy
#import highspy  # If using highs solver

pd.options.display.width = 0
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from LEAP.f_graphicalTools import *
from LEAP.f_demand_tools import *
from LEAP.f_tools import *
from LEAP.model_single_horizon_multi_energy import build_single_horizon_multi_energy_LEAP_model

sys.path.extend(['.'])

## Study setup

In [ ]:
scenario = "reference"
xls_7_nodes_file_ID = "EU_7_2050_" + scenario
# xls_7_nodes_file_ID = "EU_7_2050"

In [ ]:
# Create graphical results directory
graphical_results_folder = "case_studies/eu_7_nodes/graphical_results/"
if not os.path.exists(graphical_results_folder):
    os.makedirs(graphical_results_folder)

# Load input data (only load missing files)
input_data_folder = "case_studies/eu_7_nodes/data/"
download_input_data(input_data_folder, verbose=True)

# Step by step study

## I - Simple single area (with ramp) : 


### Loading parameters

In [ ]:
selected_area_to = ["FR"]
selected_conversion_technology = ['old_nuke', 'ccgt', 'demand_not_served']  # You'll add 'solar' after
# selected_conversion_technology = ['old_nuke', 'wind_power_on_shore', 'ccgt', 'demand_not_served', 'hydro_river', 'hydro_reservoir', 'solar']  # Try adding 'hydro_river', 'hydro_reservoir'

parameters = read_EAP_input_parameters(
    selected_area_to = selected_area_to,
    selected_conversion_technology = selected_conversion_technology,
    input_data_folder = input_data_folder,
    file_id = xls_7_nodes_file_ID,
    is_storage = False,
    is_demand_management = False
)

parameters["operation_min_1h_ramp_rate"].loc[{"conversion_technology" :"old_nuke"}] = 0.02
parameters["operation_max_1h_ramp_rate"].loc[{"conversion_technology" :"ccgt"}] = 0.05
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology" :"old_nuke"}] = 80000
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology" :"ccgt"}] = 80000


### Building model and solving the problem

In [ ]:
# Build & solve model
model = build_single_horizon_multi_energy_LEAP_model(parameters=parameters)
model.solve(solver_name='gurobi')

In [ ]:
# Synthèse Energie/Puissance/Coûts
print(extractCosts_l(model))
print()
print(extractEnergyCapacity_l(model))

In [ ]:
# Checksum (production = consumption)
assert abs(model.solution['operation_conversion_power'].sum(['conversion_technology']) - parameters['exogenous_energy_demand']).max().values < 10**-3

### Series visualization

In [ ]:
production_df = model.solution['operation_conversion_power'].to_dataframe().reset_index().pivot(
    index='date',
    columns='conversion_technology',
    values='operation_conversion_power'
)

# Build figure
fig = MyStackedPlotly(y_df=production_df, Conso=parameters["exogenous_energy_demand"].to_dataframe())
fig = fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année", autosize=True)

# Adjust the figure size
fig.update_layout(height=600)
fig.show()

## II - Addition of Storage to single area with ramp :

### Loading parameters

In [ ]:
selected_area_to = ['FR']
selected_conversion_technology = ['old_nuke', 'wind_power_on_shore', 'ocgt', 'ccgt', 'demand_not_served', 'hydro_river', 'hydro_reservoir', 'solar']
selected_storage_technology = ['storage_hydro', 'battery']


parameters = read_EAP_input_parameters(
   selected_area_to = selected_area_to,
   selected_conversion_technology = selected_conversion_technology,
   selected_storage_technology = selected_storage_technology,
   input_data_folder = input_data_folder,
   file_id = "EU_7_2050",
   is_demand_management = False
)

parameters["operation_min_1h_ramp_rate"].loc[{"conversion_technology" :"old_nuke"}] = 0.01
parameters["operation_max_1h_ramp_rate"].loc[{"conversion_technology" :"old_nuke"}] = 0.02
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology" :"old_nuke"}]=80000
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology" :"ccgt"}]=50000


### Building model and solving the problem

In [ ]:
# Build & solve model
model = build_single_horizon_multi_energy_LEAP_model(parameters=parameters)
model.solve(solver_name='gurobi')

In [ ]:
# Synthèse Energie/Puissance/Coûts
print(extractCosts_l(model))
print(extractEnergyCapacity_l(model)['Capacity_GW'])
print(extractEnergyCapacity_l(model)['Energy_TWh'])

In [ ]:
# Checksum (production = consumption)
assert abs(model.solution['operation_conversion_power'].sum(['conversion_technology']) - parameters['exogenous_energy_demand']).max().values < 10**-3

In [ ]:
Storage_production_out = model.solution['operation_storage_power_out'].rename({"storage_technology": "conversion_technology"})
Storage_production_out['conversion_technology'] = [st+"_out" for st in selected_storage_technology]
Storage_production_out.name = "operation_conversion_power"

Storage_production_in = (-model.solution['operation_storage_power_in']).rename({"storage_technology": "conversion_technology"})
Storage_production_in['conversion_technology'] = [st+"_in" for st in selected_storage_technology]
Storage_production_in.name = "operation_conversion_power"
production_xr = xr.concat([model.solution['operation_conversion_power'], Storage_production_out, Storage_production_in], dim="conversion_technology")

# Checksum (?)
assert abs(production_xr.sum(['conversion_technology']) - parameters['exogenous_energy_demand']).max() < 10**-3

### Series visualization

In [ ]:
production_df = production_xr.to_dataframe().reset_index().pivot(
    index = 'date',
    columns = 'conversion_technology',
    values = 'operation_conversion_power'
)

# Build figure
fig = MyStackedPlotly(y_df=production_df, Conso=parameters["exogenous_energy_demand"].to_dataframe())
fig = fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année", autosize=True)

# Adjust the figure size
fig.update_layout(height=600)
fig.show()

## III - Multi-zone without storage :

### Loading parameters

In [ ]:
selected_area_to = ["FR", "DE"]
selected_conversion_technology = ['old_nuke', 'ccgt', 'wind_power_on_shore', "demand_not_served"]  # You'll add 'solar' after #'new_nuke', 'hydro_river', 'hydro_reservoir','wind_power_on_shore', 'wind_power_off_shore', 'solar', 'Curtailement'}
# selected_conversion_technology = ['old_nuke', 'wind_power_on_shore', 'ccgt', "demand_not_served", 'hydro_river', 'hydro_reservoir', "solar"]  # Try adding 'hydro_river', 'hydro_reservoir'

parameters = read_EAP_input_parameters(
    selected_area_to = selected_area_to,
    selected_conversion_technology = selected_conversion_technology,
    input_data_folder = input_data_folder,
    file_id = "EU_7_2050",
    is_storage = False,
    is_demand_management = False
)

parameters["operation_min_1h_ramp_rate"].loc[{"conversion_technology": "old_nuke"}] = 0.01
parameters["operation_max_1h_ramp_rate"].loc[{"conversion_technology": "old_nuke"}] = 0.02
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology": "old_nuke"}] = 80000
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology": "ccgt"}] = 50000

### Building model and solving the problem

In [ ]:
# Build model
model = build_single_horizon_multi_energy_LEAP_model(parameters=parameters)

# Solve model
model.solve(solver_name='gurobi')
# model.solve(solver_name='highs', parallel='on')

# res = run_highs(model) #res= linopy.solvers.run_highs(model)

### Synthesis : Power / Energy / Costs

In [ ]:
print(extractCosts_l(model))
print(extractEnergyCapacity_l(model))

In [ ]:
# Checksum (production = consumption)
production_df = EnergyAndExchange2Prod(model)
assert abs(production_df.sum(axis=1) - parameters['exogenous_energy_demand'].to_dataframe()["exogenous_energy_demand"]).max() < 10**-3

### Series visualization

In [ ]:
# Build figure
fig = MyAreaStackedPlot(df_=production_df, Conso=parameters["exogenous_energy_demand"].to_dataframe())
fig = fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année", autosize=True)

# Adjust the figure size
fig.update_layout( height=600)
fig.show()

## IV - Simple single area +4 million EV +  demande side management +30TWh H2

### Loading parameters

In [ ]:
selected_area_to = ["FR", "DE"]
# selected_conversion_technology = ['old_nuke', 'ccgt', 'wind_power_on_shore', 'demand_not_served']  # You'll add 'solar' after #'new_nuke', 'hydro_river', 'hydro_reservoir','wind_power_on_shore', 'wind_power_off_shore', 'solar', 'Curtailement'}
selected_conversion_technology = ['old_nuke', 'wind_power_on_shore', 'ccgt', 'demand_not_served', 'hydro_river', 'hydro_reservoir', 'solar']  # Try adding 'hydro_river', 'hydro_reservoir'
selected_storage_technology = ['storage_hydro']

parameters = read_EAP_input_parameters(
   selected_area_to = selected_area_to,
   selected_conversion_technology = selected_conversion_technology,
   selected_storage_technology = selected_storage_technology,
   input_data_folder = input_data_folder,
   file_id = "EU_7_2050",
   is_storage = True,
   is_demand_management = True
)

parameters["operation_min_1h_ramp_rate"].loc[{"conversion_technology"   : "old_nuke"}] = 0.01
parameters["operation_max_1h_ramp_rate"].loc[{"conversion_technology"   : "old_nuke"}] = 0.02
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology": "old_nuke"}] = 80000
parameters["planning_conversion_max_capacity"].loc[{"conversion_technology": "ccgt"}] = 50000

### Building model and solving the problem

In [ ]:
# Build & solve model
model = build_single_horizon_multi_energy_LEAP_model(parameters=parameters)
model.solve(solver_name='gurobi')  # Highs not faster than cbc

### Synthesis : Power / Energy / Costs

In [ ]:
print(extractCosts_l(model))
print(extractEnergyCapacity_l(model))

In [ ]:
# Checksum (production = consumption) (?)
Prod_minus_conso = model.solution['operation_conversion_power'].sum(['conversion_technology']) - parameters['exogenous_energy_demand'] + model.solution['operation_storage_power_out'].sum(['storage_technology']) - model.solution['operation_storage_power_in'].sum(['storage_technology']) ## Storage
assert abs(Prod_minus_conso).max() < 10**-3

In [ ]:
Storage_production = (model.solution['operation_storage_power_out'] - model.solution['operation_storage_power_in']).rename({"storage_technology":"conversion_technology"})
Storage_production.name = "operation_conversion_power"

### Serie visualization

In [ ]:
production_df = production_xr.to_dataframe().reset_index().pivot(
    index="date",
    columns='conversion_technology',
    values='operation_conversion_power'
)

# Build figure
fig = MyStackedPlotly(y_df=production_df, Conso=parameters['exogenous_energy_demand'].to_dataframe())
fig = fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année", autosize=True)

# Adjust the figure size
fig.update_layout( height=600)
fig.show()

## V - 7 nodes EU model :

### Loading parameters

In [ ]:
graphical_results_folder = "case_studies/Basic_France_Germany_models/Planning_optimisation/GraphicalResults/"
# selected_conversion_technology = ['old_nuke', 'ccgt', 'wind_power_on_shore', 'demand_not_served']  # You'll add 'solar' after #'new_nuke', 'hydro_river', 'hydro_reservoir','wind_power_on_shore', 'wind_power_off_shore', 'solar', 'Curtailement'}
selected_conversion_technology = ['old_nuke', 'wind_power_on_shore', 'ccgt', 'ocgt', 'demand_not_served', 'hydro_river', 'hydro_reservoir', 'solar']  # Try adding 'hydro_river', 'hydro_reservoir'
selected_storage_technology = ['storage_hydro', 'battery']

pays = 'FR'
scenario = 'Nuke-'
parameters = read_EAP_input_parameters(
   selected_area_to = None,
   selected_conversion_technology = selected_conversion_technology,
   selected_storage_technology = selected_storage_technology,
   input_data_folder = input_data_folder,
   file_id = "EU_7_2050_" + scenario,
   is_storage = True,
   is_demand_management = True
)

parameters["operation_min_1h_ramp_rate"].loc[{"conversion_technology": "old_nuke"}] = 0.25
parameters["operation_max_1h_ramp_rate"].loc[{"conversion_technology": "old_nuke"}] = 0.25
#parameters["planning_conversion_max_capacity"].loc[{"conversion_technology": "old_nuke"}] = 80000
#parameters["planning_conversion_max_capacity"].loc[{"conversion_technology": "ccgt"}] = 50000
#parameters["planning_conversion_max_capacity"].loc[{"conversion_technology": "old_nuke"}]
parameters["flexible_demand_to_optimise"].to_dataframe().groupby(["flexible_demand", "area_to"]).sum() / 10**6

# TODO
list(parameters.keys())
year = 2018

### Building model and solving the problem

In [ ]:
# Build model
model = build_single_horizon_multi_energy_LEAP_model(parameters=parameters)

# Solve model
model.solve(solver_name='gurobi')  # gurobi: 7 minutes; highs: 24 hours
# model.solve(solver_name='highs', parallel='on')
# model.solve(solver_name='cplex')

# res = run_highs(model)  # res = linopy.solvers.run_highs(model)

### Synthesis : Power / Energy / Costs

In [ ]:
print(extractCosts_l(model))
print(extractEnergyCapacity_l(model)['Capacity_GW'])
print(extractEnergyCapacity_l(model)['Energy_TWh'])
Battery_energy_in_TWh = extractEnergyCapacity_l(model)['Energy_TWh'].loc[(slice(None), slice(None), "battery", "storage_in")]['Energy_TWh']  # TODO: KeyError: 'battery'
Battery_capacity_GW = extractEnergyCapacity_l(model)['Capacity_GW'].loc[(slice(None), slice(None), "battery", "storage_capacity")]['Capacity_GW']
(Battery_energy_in_TWh * 1000) / (Battery_capacity_GW * 4)

### Serie visualization

In [ ]:
# Checksum (production = consumption)

Storage_production_out = (model.solution['operation_storage_power_out']).rename({"storage_technology": "conversion_technology"})
Storage_production_out['conversion_technology'] = [st+"_out" for st in selected_storage_technology]
Storage_production_out.name = "operation_conversion_power"

Storage_production_in = (-model.solution['operation_storage_power_in']).rename({"storage_technology": "conversion_technology"})
Storage_production_in['conversion_technology'] = [st+"_in" for st in selected_storage_technology]
Storage_production_in.name = "operation_conversion_power"
production_xr = xr.concat([model.solution['operation_conversion_power'], Storage_production_out, Storage_production_in], dim="conversion_technology")

production_xr = xr.concat([model.solution['operation_conversion_power'],Storage_production_out, Storage_production_in], dim="conversion_technology")
production_df = EnergyAndExchange2Prod(model)


assert abs(production_df.sum(axis=1) - parameters['exogenous_energy_demand'].to_dataframe()["exogenous_energy_demand"]).max() < 10**-3

In [ ]:
iii = model.solution['exchange_op_power'].sum(['area_to']).rename({"area_from": "area_to"})
Exchange_pos = (model.solution['exchange_op_power'].sum(['area_from'])).rename({"energy_vector_out": "conversion_technology"})
Exchange_pos["conversion_technology"] = ["import"]
Exchange_neg = (-iii).rename({"energy_vector_out": "conversion_technology"})
Exchange_neg["conversion_technology"] = ["export"]

Exchange_pos.name = "operation_conversion_power"
Exchange_neg.name = "operation_conversion_power"
production_xr_ex=xr.concat([production_xr, Exchange_pos, Exchange_neg], dim="conversion_technology")


model.solution['operation_total_hourly_demand']
model.solution["planning_flexible_demand_max_power_increase"].max()

Prod_minus_conso = production_xr_ex - model.solution['operation_total_hourly_demand']  # Storage
abs(Prod_minus_conso).max()

Storage_production = (model.solution['operation_storage_power_out'] - model.solution['operation_storage_power_in']).rename({"storage_technology": "conversion_technology"})
Storage_production.name = "operation_conversion_power"
production_xr = xr.combine_by_coords([model.solution['operation_conversion_power'], Storage_production])


Storage_production.loc[{"area_to": "FR", "conversion_technology": "battery"}].min()
Storage_production.loc[{"area_to": "FR", "conversion_technology": "battery"}].max()


In [ ]:
production_df = production_xr.to_dataframe().reset_index().pivot(
    index="date",
    columns='conversion_technology',
    values='operation_conversion_power'
)

# Build figure
fig = MyStackedPlotly(y_df=production_df, Conso=parameters['exogenous_energy_demand'].to_dataframe())
fig = fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année", autosize=True)

# Adjust the figure size
fig.update_layout(height=600)
fig.show()

# Appendix

In [ ]:
# Sur le nombre de contraintes
A_T = 4; A_ST = 3; D_A = 2; D_A_T = 4; D_A_ST = 5
A = 7 ; ST =2 ; D = 8760 ; T = 10

A*T*A_T + A*ST*A_ST + D*A*D_A + D*A*T*D_A_T + D*A*ST*D_A_ST

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Données fournies
xvals = [0,1E-04,0.01,	0.02,	0.03,	0.04,	0.05,	0.06,	0.07,	0.08,	0.09,	0.1]
x_vals=[x*100 for x in xvals]
y_vals = [x+1 for x in range(80)]
import pandas as pd
import numpy as np

# Définir la fonction f(x, y)
def Lr(r, L):
    return ((1+r/100)**L -1)/(r/100*(r/100+1)**L)

# Générer des valeurs pour x, y
x_values = np.linspace(1, 10, 10)  # Exemple de valeurs pour x
y_values = np.linspace(1, 80, 110)  # Exemple de valeurs pour y

# Créer une grille 2D pour x et y
x, y = np.meshgrid(x_values, y_values)

# Appliquer la fonction f à la grille
z = Lr(x, y).ravel()

# Créer un DataFrame avec un double index construit avec x et y
df = pd.DataFrame({'Corrected_LL': z}, index=pd.MultiIndex.from_product([x_values, y_values], names=['taux', 'Life_length']))

fig = go.Figure()

# Ajouter une surface 3D avec x, y et z
contour = go.Contour(x=x_values, y=y_values, z=df['Corrected_LL'].values.reshape(x.shape), colorscale='Viridis', line=dict(smoothing=0.85), contours=dict(showlabels=True))
fig.add_trace(contour)

# Ajouter l'axe x, y et z
fig.update_layout(
    xaxis_title='Taux d\'actualisation en %',
    yaxis_title='Durée de vie en années',
    title='Lignes de niveau discrètes de la durée de vie corrigée avec Plotly'
)
fig.show()